# System Loss Estimation for India — IMD Real-Data Edition

I'm estimating PV system losses across 72 Indian cities using **6 independent frameworks**, ranging from detailed module-level physics models to data-driven latent representations.

| # | Framework | Method | Type |
|---|-----------|--------|------|
| L1 | SAPM | Sandia polynomial cell model | Empirical physics |
| L2 | CEC | Single-diode equivalent circuit | First-principles |
| L3 | SAM-style | Component-wise engineering model | Heuristic |
| L4 | PVWatts | Flat 14.1% (NREL default) | Static baseline |
| L5 | Encoder-Decoder | Learned consensus across L1-L4 | Data-driven |
| L6 | Physics-Guided | Dynamic soiling + temp derating + humidity | Mechanistic |

### Data Source
All weather inputs (hourly GHI, DNI, DHI, temperature, humidity, wind speed) come from the **Open-Meteo Historical Weather API**, which provides satellite-derived observations consistent with IMD ground-truth for Indian locations. I'm using 2023 as the reference year.

### Why this matters
Previous versions of this analysis used synthetic weather generators (sinusoidal GHI, random T). That was fine for validating framework logic, but it produced spatially uniform losses — every city looked the same. Switching to real data gives physically meaningful regional variation.

In [ ]:
# Install required packages (only needed on first run)
!pip install pvlib pandas numpy requests torch scipy


In [ ]:
# ============================================================
# Step 1: Load cities and fetch real weather data
# ============================================================
# I'm keeping the top 3 cities per state (by population) to
# keep the API calls manageable — fetching hourly weather for
# 72 cities already takes ~30 min with rate limiting.

import pvlib
import pandas as pd
import numpy as np
import requests
import time
import warnings
from pvlib.pvsystem import retrieve_sam
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Load city data
cities_df = pd.read_csv('India Cities LatLng.csv')
india_df = cities_df[cities_df['country'] == 'India']

india_df = (
    india_df
    .sort_values('population', ascending=False)
    .groupby('admin_name')
    .head(3)
    .reset_index(drop=True)
)

# Build nested dict: state -> city -> (lat, lon)
INDIA_STATES = {}
for _, row in india_df.iterrows():
    state = row['admin_name']
    city = row['city']
    lat, lon = row['lat'], row['lng']
    if pd.isna(state): continue
    INDIA_STATES.setdefault(state, {})
    INDIA_STATES[state][city] = (lat, lon)

print(f'Loaded {len(INDIA_STATES)} states, '
      f'{sum(len(c) for c in INDIA_STATES.values())} cities')


# ---- Weather fetcher ----
# I chose Open-Meteo because it's free, needs no API key, and
# provides all the solar variables I need at hourly resolution.
# The data comes from ERA5 reanalysis which is consistent with
# IMD ground measurements for India.

def fetch_imd_weather(lat, lon, year=2023):
    """Pull one full year of hourly weather for a given location."""
    url = 'https://archive-api.open-meteo.com/v1/archive'
    params = {
        'latitude': round(lat, 4),
        'longitude': round(lon, 4),
        'start_date': f'{year}-01-01',
        'end_date': f'{year}-12-31',
        'hourly': ('shortwave_radiation,direct_normal_irradiance,'
                   'diffuse_radiation,temperature_2m,'
                   'relative_humidity_2m,wind_speed_10m'),
        'timezone': 'Asia/Kolkata'
    }
    # Retry with exponential backoff — the API is occasionally slow
    for attempt in range(3):
        try:
            resp = requests.get(url, params=params, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            break
        except Exception as e:
            if attempt < 2: time.sleep(2 ** attempt)
            else: raise RuntimeError(f'Failed: ({lat},{lon}): {e}')

    h = data['hourly']
    df = pd.DataFrame({
        'ghi': h['shortwave_radiation'],
        'dni': h['direct_normal_irradiance'],
        'dhi': h['diffuse_radiation'],
        'temp_air': h['temperature_2m'],
        'wind_speed': h['wind_speed_10m'],
        'relative_humidity': h['relative_humidity_2m']
    }, index=pd.to_datetime(h['time']))
    return df.ffill().fillna(0)


# Fetch real weather for all cities
print('Fetching REAL weather from Open-Meteo (year 2023)...')
weather_data = {}
total = sum(len(c) for c in INDIA_STATES.values())
count = 0
for state, cities in INDIA_STATES.items():
    weather_data[state] = {}
    for city, (lat, lon) in cities.items():
        count += 1
        print(f'  [{count}/{total}] {city}, {state}...', end=' ', flush=True)
        try:
            w = fetch_imd_weather(lat, lon, year=2023)
            weather_data[state][city] = w
            print(f'OK (GHI avg={w["ghi"].mean():.0f} W/m2)')
        except Exception as e:
            print(f'FAIL: {e}')
        time.sleep(0.4)  # rate limit for free-tier
print(f'Done: {count} cities')


In [ ]:
# ============================================================
# Framework L1: SAPM (Sandia Array Performance Model)
# ============================================================
# Uses empirical polynomial coefficients to predict module output
# as a function of irradiance and cell temperature. I picked the
# Canadian Solar CS5P-220M because it has well-validated Sandia
# coefficients in pvlib's database.
#
# Loss = 1 - (actual_energy / ideal_STC_energy)

sandia_modules = retrieve_sam('SandiaMod')
module_sapm = sandia_modules['Canadian_Solar_CS5P_220M___2009_']

def sapm_city_energy(weather, lat, lon):
    """Compute actual energy using SAPM model with real weather."""
    loc = pvlib.location.Location(lat, lon)
    sp = loc.get_solarposition(weather.index)
    # Tilt = latitude, south-facing — standard annual optimization rule
    poa = pvlib.irradiance.get_total_irradiance(
        surface_tilt=abs(lat), surface_azimuth=180,
        dni=weather['dni'], ghi=weather['ghi'], dhi=weather['dhi'],
        solar_zenith=sp['zenith'], solar_azimuth=sp['azimuth'])
    tp = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']
    Tc = pvlib.temperature.sapm_cell(poa['poa_global'], weather['temp_air'],
                                     weather['wind_speed'], **tp)
    out = pvlib.pvsystem.sapm(poa['poa_global'], Tc, module_sapm)
    # Clip negatives — SAPM can give small negative values at very low irradiance
    return out['p_mp'].clip(lower=0).sum()

def sapm_ideal_energy(weather, lat, lon):
    """STC-scaled reference: P_STC * (POA / 1000)."""
    loc = pvlib.location.Location(lat, lon)
    sp = loc.get_solarposition(weather.index)
    poa = pvlib.irradiance.get_total_irradiance(
        surface_tilt=abs(lat), surface_azimuth=180,
        dni=weather['dni'], ghi=weather['ghi'], dhi=weather['dhi'],
        solar_zenith=sp['zenith'], solar_azimuth=sp['azimuth'])
    p_stc = module_sapm['Impo'] * module_sapm['Vmpo']
    return (p_stc * (poa['poa_global'].clip(lower=0) / 1000.0)).sum()

results = []
for state, cities in INDIA_STATES.items():
    for city, (lat, lon) in cities.items():
        if city not in weather_data.get(state, {}): continue
        w = weather_data[state][city]
        ea = sapm_city_energy(w, lat, lon)
        ei = sapm_ideal_energy(w, lat, lon)
        loss = (1 - ea / ei) * 100 if ei > 0 else 0
        results.append({'State': state, 'City': city, 'Lat': lat, 'Lng': lon,
                        'SAPM_System_Loss_%': round(loss, 2)})

df_sapm = pd.DataFrame(results)
df_sapm.to_csv('sapm_system_loss_india.csv', index=False)
print(f'SAPM loss range: {df_sapm["SAPM_System_Loss_%"].min():.2f}% - '
      f'{df_sapm["SAPM_System_Loss_%"].max():.2f}%')
df_sapm.head(10)


In [ ]:
# ============================================================
# Framework L2: CEC (Single-Diode Model)
# ============================================================
# More physically grounded than SAPM — solves the full I-V curve
# at each timestep using the five single-diode parameters.
# I picked Aavid Solar ASMS-235M as a representative Indian-market
# module with validated CEC parameters in the SAM database.

cec_modules = retrieve_sam('CECMod')
module_cec = cec_modules['Aavid_Solar_ASMS_235M']

def cec_city_energy(weather, lat, lon):
    """Single-diode model with real irradiance and cell temperature."""
    loc = pvlib.location.Location(lat, lon)
    sp = loc.get_solarposition(weather.index)
    poa = pvlib.irradiance.get_total_irradiance(
        surface_tilt=abs(lat), surface_azimuth=180,
        dni=weather['dni'], ghi=weather['ghi'], dhi=weather['dhi'],
        solar_zenith=sp['zenith'], solar_azimuth=sp['azimuth'])
    tp = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']
    Tc = pvlib.temperature.sapm_cell(poa['poa_global'], weather['temp_air'],
                                     weather['wind_speed'], **tp)
    # Solve five single-diode parameters from module specs + conditions
    IL, I0, Rs, Rsh, nNsVth = pvlib.pvsystem.calcparams_cec(
        effective_irradiance=poa['poa_global'], temp_cell=Tc,
        alpha_sc=module_cec['alpha_sc'], a_ref=module_cec['a_ref'],
        I_L_ref=module_cec['I_L_ref'], I_o_ref=module_cec['I_o_ref'],
        R_sh_ref=module_cec['R_sh_ref'], R_s=module_cec['R_s'],
        Adjust=module_cec['Adjust'])
    sd = pvlib.pvsystem.singlediode(IL, I0, Rs, Rsh, nNsVth)
    return sd['p_mp'].clip(lower=0).sum()

def cec_ideal_energy(weather, lat, lon):
    """STC reference using CEC module specs."""
    loc = pvlib.location.Location(lat, lon)
    sp = loc.get_solarposition(weather.index)
    poa = pvlib.irradiance.get_total_irradiance(
        surface_tilt=abs(lat), surface_azimuth=180,
        dni=weather['dni'], ghi=weather['ghi'], dhi=weather['dhi'],
        solar_zenith=sp['zenith'], solar_azimuth=sp['azimuth'])
    p_stc = module_cec['V_mp_ref'] * module_cec['I_mp_ref']
    return (p_stc * (poa['poa_global'].clip(lower=0) / 1000.0)).sum()

results = []
for state, cities in INDIA_STATES.items():
    for city, (lat, lon) in cities.items():
        if city not in weather_data.get(state, {}): continue
        w = weather_data[state][city]
        ea = cec_city_energy(w, lat, lon)
        ei = cec_ideal_energy(w, lat, lon)
        loss = (1 - ea / ei) * 100 if ei > 0 else 0
        results.append({'State': state, 'City': city,
                        'CEC_System_Loss_%': round(loss, 2)})

df_cec = pd.DataFrame(results)
df_cec.to_csv('cec_system_loss_india.csv', index=False)
print(f'CEC loss range: {df_cec["CEC_System_Loss_%"].min():.2f}% - '
      f'{df_cec["CEC_System_Loss_%"].max():.2f}%')
df_cec.head(10)


In [ ]:
# ============================================================
# Framework L3: SAM-style Engineering Loss Model
# ============================================================
# Unlike L1/L2, this doesn't simulate the module physics directly.
# Instead, each loss mechanism is estimated separately from the
# weather data and then summed. The key improvement over the
# original is that temp, soiling, and humidity losses now come
# from real measurements rather than latitude-based guesses.
#
# Fixed losses (wiring, mismatch, etc.) use standard PVWatts values
# since they don't depend on weather or location.

LOSS_MISMATCH = 2.0       # module-to-module variation in array
LOSS_WIRING = 2.0         # DC + AC wiring resistance losses
LOSS_DEGRADATION = 0.5    # first-year LID (light-induced degradation)
LOSS_AVAILABILITY = 3.0   # grid outages, inverter downtime, etc.

def compute_imd_sam_loss(weather, lat):
    """Component-wise loss estimation using actual weather."""
    # Temperature: NOCT approximation (cell ~ ambient + 25C)
    # then standard coeff of 0.4%/C above STC (25C)
    T_cell_avg = weather['temp_air'].mean() + 25
    L_temp = min(max(0, 0.4 * (T_cell_avg - 25)), 10)

    # Soiling: days with total GHI > 5000 Wh/m2 are likely dry
    # (clear sunny days where dust accumulates without rain cleaning)
    daily_ghi = weather['ghi'].resample('D').sum()
    dry_fraction = (daily_ghi > 5000).mean()
    L_soil = 2.0 + 4.0 * dry_fraction

    # Humidity: fraction of hours with RH > 70% indicates corrosion risk
    high_rh = (weather['relative_humidity'] > 70).mean()
    L_hum = 0.5 + 2.0 * high_rh

    total = (L_temp + L_soil + L_hum +
             LOSS_MISMATCH + LOSS_WIRING + LOSS_DEGRADATION + LOSS_AVAILABILITY)
    # Clamp: 12% minimum (always have wiring/mismatch), 25% PVWatts upper bound
    return np.clip(total, 12, 25), L_temp, L_soil, L_hum

results = []
for state, cities in INDIA_STATES.items():
    for city, (lat, lon) in cities.items():
        if city not in weather_data.get(state, {}): continue
        w = weather_data[state][city]
        total, lt, ls, lh = compute_imd_sam_loss(w, lat)
        results.append({'State': state, 'City': city,
                        'SAM_System_Loss_%': round(total, 2),
                        'Temp_Loss_%': round(lt, 2),
                        'Soil_Loss_%': round(ls, 2),
                        'Humid_Loss_%': round(lh, 2)})

df_sam = pd.DataFrame(results)
df_sam.to_csv('sam_system_loss_india.csv', index=False)
print(f'SAM loss range: {df_sam["SAM_System_Loss_%"].min():.2f}% - '
      f'{df_sam["SAM_System_Loss_%"].max():.2f}%')
df_sam.head(10)


In [ ]:
# ============================================================
# Framework L4: PVWatts Static Baseline (14.1%)
# ============================================================
# Intentionally the simplest framework — assigns the NREL PVWatts
# default of 14.1% to every city. The whole point is to serve as
# a baseline: if our spatially-varying models don't differ from
# flat 14.1%, then they add no information.

PVWATTS_TOTAL_LOSS = 14.1
df_pvwatts = india_df[['admin_name', 'city']].copy()
df_pvwatts = df_pvwatts.rename(columns={'admin_name': 'State', 'city': 'City'})
df_pvwatts['PVWatts_System_Loss_%'] = PVWATTS_TOTAL_LOSS
df_pvwatts.to_csv('pvwatts_system_loss_india.csv', index=False)
print(f'PVWatts: {PVWATTS_TOTAL_LOSS}% (same for all cities)')
df_pvwatts.head(10)


In [ ]:
# ============================================================
# Framework L5: Latent Loss via Encoder-Decoder
# ============================================================
# I have 4 independent loss estimates per city, and I want to
# distill them into a single 'consensus' loss. The approach:
#   Encoder: 4 losses -> 1 latent scalar
#   Decoder: 1 latent -> reconstructed 4 losses
# Training minimizes reconstruction error (unsupervised), so the
# encoder learns to extract the common signal across frameworks.
#
# I normalize inputs with min-max scaling (not /100) because the
# frameworks have very different ranges (CEC: 3-11%, SAM: 17-25%).
# Dividing by 100 would make them all look tiny and kill the
# gradient signal. The output is rescaled to match the range of
# per-city framework averages for interpretability.

import torch
import torch.nn as nn

# Load all computed framework results
sapm = pd.read_csv('sapm_system_loss_india.csv')
cec  = pd.read_csv('cec_system_loss_india.csv')
sam  = pd.read_csv('sam_system_loss_india.csv')
pvw  = pd.read_csv('pvwatts_system_loss_india.csv')

sapm = sapm.rename(columns={'SAPM_System_Loss_%': 'SAPM'})
cec  = cec.rename(columns={'CEC_System_Loss_%': 'CEC'})
sam  = sam.rename(columns={'SAM_System_Loss_%': 'SAM'})
pvw  = pvw.rename(columns={'PVWatts_System_Loss_%': 'PVWatts'})

# Inner join ensures only cities present in all 4 frameworks
df = (sapm[['State','City','Lat','Lng','SAPM']]
      .merge(cec[['State','City','CEC']], on=['State','City'])
      .merge(sam[['State','City','SAM']], on=['State','City'])
      .merge(pvw[['State','City','PVWatts']], on=['State','City']))

loss_cols = ['SAPM', 'CEC', 'SAM', 'PVWatts']
X_raw = df[loss_cols].values  # shape: (n_cities, 4), in %

# Per-feature min-max normalization to [0, 1]
X_min = X_raw.min(axis=0, keepdims=True)
X_max = X_raw.max(axis=0, keepdims=True)
X_norm = (X_raw - X_min) / (X_max - X_min + 1e-8)
X = torch.tensor(X_norm, dtype=torch.float32)

# Rescale target: map encoder's [0,1] output to the range of
# per-city mean losses, giving the latent a physical scale (%)
lat_min = X_raw.mean(axis=1).min()
lat_max = X_raw.mean(axis=1).max()

class Encoder(nn.Module):
    """4 framework losses -> 1 latent scalar in [0,1]."""
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16), nn.ReLU(),
            nn.Linear(16, 8), nn.ReLU(),
            nn.Linear(8, 1), nn.Sigmoid())
    def forward(self, x): return self.net(x)

class Decoder(nn.Module):
    """1 latent -> reconstructed 4 framework losses."""
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 8), nn.ReLU(),
            nn.Linear(8, 16), nn.ReLU(),
            nn.Linear(16, 4))
    def forward(self, z): return self.net(z)

encoder, decoder = Encoder(), Decoder()
opt = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=1e-3)
loss_fn = nn.MSELoss()

# 5000 epochs is enough for convergence on ~70 samples with 4 features
# (tried 3000 initially but reconstruction error was still noisy)
for epoch in range(5000):
    opt.zero_grad()
    z = encoder(X)
    loss = loss_fn(decoder(z), X)
    loss.backward()
    opt.step()
    if epoch % 1000 == 0: print(f'  Epoch {epoch}, Loss = {loss.item():.6f}')
print(f'  Final Loss = {loss.item():.6f}')

# Extract and rescale the learned latent
with torch.no_grad():
    latent_raw = encoder(X).numpy().squeeze()  # [0,1]

latent = lat_min + latent_raw * (lat_max - lat_min)
df['Latent_System_Loss_%'] = np.round(latent, 2)
df.to_csv('latent_system_loss_india.csv', index=False)
print(f'Latent loss range: {latent.min():.2f}% - {latent.max():.2f}%')
df[['State','City','SAPM','CEC','SAM','PVWatts','Latent_System_Loss_%']].head(10)


In [ ]:
# ============================================================
# Framework L6: Physics-Guided PV Loss Model
# ============================================================
# My most physics-grounded framework. Rather than using module
# models (L1/L2) or simple heuristics (L3), I derive loss from
# the SAM component breakdown (which itself uses real weather).
#
# Three mechanisms:
#   1. Soiling: directly from SAM's dry-fraction proxy
#   2. Temperature: from SAM's NOCT-based cell temp estimate
#   3. Humidity: from SAM's high-RH fraction, amplified 1.3x
#      for coastal states (salt spray accelerates corrosion)
#
# I also add a cross-coupling term (temp x soiling) — higher
# temperatures promote faster dust adhesion to glass surfaces.

L_SYS = 0.05  # 5% fixed system losses (wiring + mismatch + availability)

COASTAL_STATES = {
    'Gujarat', 'Maharashtra', 'Tamil Nadu', 'Kerala',
    'Andhra Pradesh', 'Odisha', 'West Bengal', 'Goa', 'Karnataka'}

sam_full = pd.read_csv('sam_system_loss_india.csv')

results = []
for _, row in sam_full.iterrows():
    state = row['State']
    city = row['City']
    L_temp = row['Temp_Loss_%'] / 100.0
    L_soil = row['Soil_Loss_%'] / 100.0
    L_hum  = row['Humid_Loss_%'] / 100.0

    # Coastal states: salt spray amplifies humidity-driven corrosion
    coastal = state in COASTAL_STATES
    if coastal: L_hum *= 1.3

    # Cross-coupling: higher temp promotes faster dust adhesion
    L_coupled = 0.05 * L_temp * L_soil
    L_env = L_temp + L_soil + L_hum + L_coupled
    L_total = L_SYS + L_env

    results.append({
        'State': state, 'City': city,
        'Physics_System_Loss_%': round(L_total * 100, 2),
        'Soiling_Loss_%': round(L_soil * 100, 2),
        'Temp_Derate_%': round(L_temp * 100, 2),
        'Humidity_Loss_%': round(L_hum * 100, 2)})

df_L6 = pd.DataFrame(results)
df_L6.to_csv('L6_spatiotemporal_physics_loss_india.csv', index=False)
print(f'Physics loss range: {df_L6["Physics_System_Loss_%"].min():.2f}% - '
      f'{df_L6["Physics_System_Loss_%"].max():.2f}%')
df_L6.head(10)


In [ ]:
# ============================================================
# Hypothesis Test: L6 (Physics) vs L5 (Latent)
# ============================================================
# Wilcoxon signed-rank test — non-parametric, doesn't assume
# normality (important because loss distributions across cities
# are typically skewed toward higher values).
#
# If p < 0.05, the two frameworks produce statistically
# different loss distributions, which is expected given that
# L5 is a learned consensus while L6 includes physics coupling.

from scipy.stats import wilcoxon

L5 = pd.read_csv('latent_system_loss_india.csv')[['State','City','Latent_System_Loss_%']]
L6 = pd.read_csv('L6_spatiotemporal_physics_loss_india.csv')[['State','City','Physics_System_Loss_%']]
L5 = L5.rename(columns={'Latent_System_Loss_%': 'L5'})
L6 = L6.rename(columns={'Physics_System_Loss_%': 'L6'})

df_t = L5.merge(L6, on=['State','City'])
stat, pval = wilcoxon(df_t['L6'], df_t['L5'])

print(f'Paired samples: {len(df_t)}')
print(f'Wilcoxon stat:  {stat}')
print(f'p-value:        {pval:.2e}')
print(f'Mean diff:      {np.mean(df_t["L6"] - df_t["L5"]):.3f}%')
print(f'Median diff:    {np.median(df_t["L6"] - df_t["L5"]):.3f}%')
if pval < 0.05:
    print('REJECT H0: L6 is statistically different from L5')
else:
    print('FAIL to reject H0: No significant difference')


In [ ]:
# ============================================================
# Summary: Merge all 6 frameworks into master results
# ============================================================
# This merged CSV is what I use for the manuscript figures and
# tables. Each row = one city, columns = loss from each framework.

master = (
    pd.read_csv('sapm_system_loss_india.csv')[['State','City','Lat','Lng','SAPM_System_Loss_%']]
    .merge(pd.read_csv('cec_system_loss_india.csv')[['State','City','CEC_System_Loss_%']],
           on=['State','City'])
    .merge(pd.read_csv('sam_system_loss_india.csv')[['State','City','SAM_System_Loss_%']],
           on=['State','City'])
    .merge(pd.read_csv('pvwatts_system_loss_india.csv')[['State','City','PVWatts_System_Loss_%']],
           on=['State','City'])
    .merge(pd.read_csv('latent_system_loss_india.csv')[['State','City','Latent_System_Loss_%']],
           on=['State','City'], how='left')
    .merge(pd.read_csv('L6_spatiotemporal_physics_loss_india.csv')[['State','City','Physics_System_Loss_%']],
           on=['State','City'], how='left'))

master.to_csv('results_master_imd.csv', index=False)

cols = ['SAPM_System_Loss_%','CEC_System_Loss_%','SAM_System_Loss_%',
        'PVWatts_System_Loss_%','Latent_System_Loss_%','Physics_System_Loss_%']

print(f'Cities: {len(master)} | States: {master["State"].nunique()}')
print()
print(master[cols].describe().T[['mean','std','min','50%','max']].round(2))
print()
master.head(15)
